In [9]:
import pytesseract
from PIL import Image
import pandas as pd
import re
import os
import cv2
import pytesseract
from PIL import Image
import pandas as pd
import re
import numpy as np
from scipy.ndimage import interpolation as inter

# Configuration for tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\nirma\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'


def extract_text_from_image(image_path):
    image=cv2.imread(image_path,-1)
    #image = preprocess_image(image_path, 15, 8)
    angle, corrected = correct_skew(image)
    #print('Skew angle:', angle)
    #cv2.imshow(corrected)   
    text = pytesseract.image_to_string(corrected)
    #print(text)
    return text       

def parse_parameters(imgname,text):
    # Add the blood parameters you want to extract here
    parameters = ['RBC','RBC Count','Platelets', 'HB', 'Hemoglobin', 'PLT','WBC','Neutrophils','Lymphocytes']
    data = {}
    if "neutrophil" in text.lower():
                print(imgname)
    data['image']=imgname
    for param in parameters:
        match = re.search(f"{param}[:\s]+([\d.]+)", text, re.IGNORECASE)
        if match:
            data[param] = float(match.group(1))
        else:
            data[param] = None
    return data

def extract_data_from_images(image_folder):
    data = []
    for filename in os.listdir(image_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_folder, filename)
            text = extract_text_from_image(image_path)
            data.append(parse_parameters(filename,text))
    return data

def save_data_to_csv(data, output_file):
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)


def correct_skew(image, delta=1, limit=5):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1, dtype=float)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2, dtype=float)
        return histogram, score

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    corrected = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC,borderMode=cv2.BORDER_REPLICATE)
    #cv2.imshow('corrected', corrected)
    return best_angle, corrected
   
    
if __name__ == '__main__':
    image_folder = 'Pathology Reports'
    output_file = 'output.csv'
    data = extract_data_from_images(image_folder)
    save_data_to_csv(data, output_file)

10.png
11.jpg
15.jpg
16.jpeg
18.jpg
19.jpeg
23.jpg
24.jpeg
25.png
27.jpg
3.png
35.jpg
6.jpg
7.jpg
8.jpg
9.jpg
aada.png
Haematology_-_Blood_Count_1498125772.jpg
Image_2.jpg
Image_21.jpg
Image_25.jpg
Image_30.jpg
Image_35.jpg
Image_5.jpg
Image_7.jpg
Image_75.jpg


In [4]:
data

[{'image': '1.jpg',
  'RBC': None,
  'RBC Count': None,
  'Platelets': None,
  'HB': None,
  'Hemoglobin': None,
  'PLT': None,
  'WBC': None,
  'Neutrophils': None,
  'Lymphocytes': None},
 {'image': '10.png',
  'RBC': None,
  'RBC Count': None,
  'Platelets': None,
  'HB': None,
  'Hemoglobin': None,
  'PLT': None,
  'WBC': None,
  'Neutrophils': None,
  'Lymphocytes': None},
 {'image': '11.jpg',
  'RBC': None,
  'RBC Count': None,
  'Platelets': None,
  'HB': None,
  'Hemoglobin': None,
  'PLT': None,
  'WBC': None,
  'Neutrophils': None,
  'Lymphocytes': 22.3},
 {'image': '14.jpeg',
  'RBC': None,
  'RBC Count': None,
  'Platelets': None,
  'HB': None,
  'Hemoglobin': None,
  'PLT': None,
  'WBC': None,
  'Neutrophils': None,
  'Lymphocytes': 21.0},
 {'image': '15.jpg',
  'RBC': None,
  'RBC Count': None,
  'Platelets': None,
  'HB': None,
  'Hemoglobin': None,
  'PLT': None,
  'WBC': None,
  'Neutrophils': None,
  'Lymphocytes': None},
 {'image': '16.jpeg',
  'RBC': None,
  'RBC C

In [54]:
import cv2
import numpy as np
import random
import os

# Rotation
def rotate(image, angle):
    # Calculate image center
    center = tuple(np.array(image.shape[1::-1]) / 2)
    # Build the rotation matrix
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    # Perform the actual rotation and return the result
    return cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR)

def add_noise(image, scale=0.1):
    """
    Add random noise to the input image.
    """
    # Generate random noise and scale it based on the input image
    noise = np.random.normal(scale=scale, size=image.shape)
    # Add the noise to the image
    noisy_image = image + noise
    # Clip the pixel values to be within the valid range [0, 255]
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image

def blur(image, kernel_size=3):
    """
    Apply Gaussian blur to the input image.
    """
    # Apply Gaussian blur to the image using the specified kernel size
    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    return blurred_image

def grayscale(image):
    if image.shape[-1] > 1:
        result = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        result = image
    return result

def color_hue(image, hue):
    if image.shape[-1] >= 3:
        # Convert the image to HSV color space
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        # Add the hue value to the Hue channel
        hsv[:, :, 0] = (hsv[:, :, 0] + hue) % 180
        # Convert the image back to BGR color space
        result = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    else:
        result = image
    return result

def skew(image, angle=1, scale=1.0):
    """
    Skew the input image by a given angle.
    """
    # Define the transformation matrix
    matrix = np.float32([[1, 0, 0], [0, 1, 0]])
    angle_radians = angle * np.pi / 180.0
    matrix[0, 1] = np.tan(angle_radians)
    matrix[1, 0] = np.tan(angle_radians)
    # Apply the transformation to the image
    skewed_image = cv2.warpAffine(image, matrix, (image.shape[1], image.shape[0]))
    return skewed_image

def translate_x(image, shift=10):
    """
    Translate the input image on the x-axis by a given number of pixels.
    """
    # Define the transformation matrix
    matrix = np.float32([[1, 0, shift], [0, 1, 0]])
    # Apply the transformation to the image
    translated_image = cv2.warpAffine(image, matrix, (image.shape[1], image.shape[0]))
    return translated_image

def translate_y(image, shift=10):
    """
    Translate the input image on the y-axis by a given number of pixels.
    """
    # Define the transformation matrix
    matrix = np.float32([[1, 0, 0], [0, 1, shift]])
    # Apply the transformation to the image
    translated_image = cv2.warpAffine(image, matrix, (image.shape[1], image.shape[0]))
    return translated_image

# Apply one or more image variations to an input image and save the result
def apply_image_variations(img, output_path, variations):
    # Load the input image
    input_image = img

    # Apply each specified variation to the image
    for variation in variations:
        if variation == 'rotate_left':
            input_image = rotate(input_image, random_float())
        elif variation == 'rotate_right':
            input_image = rotate(input_image, -1 * random_float())
        elif variation == 'add_noise':
            input_image = add_noise(input_image)
        elif variation == 'blur':
            input_image = blur(input_image)
        elif variation == 'grayscale':
            input_image = grayscale(input_image)
        elif variation == 'color_hue':
            input_image = color_hue(input_image, 30)
        elif variation == 'translate_x':
            input_image = translate_x(input_image)
        elif variation == 'translate_y':
            input_image = translate_y(input_image)
        elif variation == 'skew':
            input_image = skew(input_image, random_float())

    # Save the resulting image to the specified output path
    cv2.imwrite(output_path, input_image)

def get_random_elements():
    input_list = ['add_noise', 'color_hue', 'skew', 'translate_y', 'translate_x']
    num_elements = random.randint(1, len(input_list))
    random.shuffle(input_list)
    return input_list[:num_elements]

def random_float():
    return random.uniform(-3, 3)

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append([filename,img])
    return images

def main():
    FOLDER_NAME = 'Pathology Reports'
    for [filename, image] in load_images_from_folder(FOLDER_NAME):
        for i in range(20):
            variations = get_random_elements()
            output_filename = f"Edited/edited-{filename}_{'-'.join(variations)}.jpg"
            print(f"[Output]: {output_filename}")
            apply_image_variations(image, output_filename, variations)

In [55]:
main()

[Output]: Edited/edited-1.jpg_color_hue-translate_x-add_noise.jpg
[Output]: Edited/edited-1.jpg_skew-color_hue-translate_y-translate_x.jpg
[Output]: Edited/edited-1.jpg_translate_x-add_noise-skew-color_hue.jpg
[Output]: Edited/edited-1.jpg_color_hue-add_noise.jpg
[Output]: Edited/edited-1.jpg_translate_x-add_noise-translate_y-color_hue-skew.jpg
[Output]: Edited/edited-1.jpg_color_hue-add_noise-translate_x-skew.jpg
[Output]: Edited/edited-1.jpg_skew-color_hue.jpg
[Output]: Edited/edited-1.jpg_color_hue-translate_x-skew-add_noise.jpg
[Output]: Edited/edited-1.jpg_skew-color_hue-translate_x-translate_y-add_noise.jpg
[Output]: Edited/edited-1.jpg_skew-translate_x.jpg
[Output]: Edited/edited-1.jpg_translate_y-color_hue-translate_x-skew.jpg
[Output]: Edited/edited-1.jpg_add_noise-translate_x-color_hue-skew-translate_y.jpg
[Output]: Edited/edited-1.jpg_translate_y.jpg
[Output]: Edited/edited-1.jpg_add_noise-skew-color_hue-translate_x-translate_y.jpg
[Output]: Edited/edited-1.jpg_skew-color_hu

In [1]:
import random
import os


def split_dataset(image_files, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    # shuffle the image files
    random.shuffle(image_files)

    # calculate the number of images for each set
    num_train = int(len(image_files) * train_ratio)
    num_val = int(len(image_files) * val_ratio)
    num_test = int(len(image_files) * test_ratio)

    # split the image files into the sets
    train_files = image_files[:num_train]
    val_files = image_files[num_train:num_train+num_val]
    test_files = image_files[num_train+num_val:]

    return train_files, val_files, test_files


In [3]:
# Set folder path
folder_path = 'meh'

# Get list of image files in folder
image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.jpg')]

# Shuffle image files
random.shuffle(image_files)

# Split image files into train and test sets
train_split = 0.7
num_train = int(len(image_files) * train_split)
train_files = image_files[:num_train]
test_files = image_files[num_train:]

# Print number of files in train and test sets
print('Number of training files:', len(train_files))
print('Number of testing files:', len(test_files))


Number of training files: 51
Number of testing files: 22
